In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [15]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/ML_cars/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/matrix/ML_cars/dw_matrix_car


In [16]:
ls

data/  day1_cars_ML.ipynb  day2_cars_matrix.ipynb  Matrix_Day3.ipynb  README.md


In [0]:
df = pd.read_hdf('data/car.h5')

In [66]:
df.head()

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,...,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
0,"[Osobowe, Dacia, Logan, Dacia Logan II (2012-)]","13:23, 5 marca 2018",PLN,Cena Brutto,45100.0,"Krasne 7A - 36-007 Krasne, rzeszowski, Podkarp...",AUTO SPEKTRUM,Dealer,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,...,None,None,Firmy,90 KM,None,None,None,None,None,Benzyna+LPG,None,None,None,None,None,None,None,Logan,898 cm3,None,None,None,None,None,None,None,None,None,Inny kolor,None,None,Nowe,II (2012-),None,None,None,Tak,None,None,0
2,"[Osobowe, Peugeot, 3008, Peugeot 3008 I (2009-...","15:48, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji, Faktura VAT",28000.0,"Tomaszów Mazowiecki, tomaszowski, Łódzkie",None,Osoba prywatna,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,...,None,Tak,Osoby prywatnej,115 KM,None,Polska,None,None,None,Diesel,None,None,None,None,None,None,None,3008,1 560 cm3,None,None,None,None,None,None,None,None,None,Czarny,None,None,Używane,I (2009-2016),None,None,None,Tak,None,None,2
3,"[Osobowe, Porsche, Cayenne, Porsche Cayenne II...","06:30, 4 marca 2018",PLN,"Cena Netto, Do negocjacji, Faktura VAT",229500.0,"Kępińska 24a - 63-640 Bralin, kępiński, Wielko...",P.W. GRIB,Dealer,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,False,True,True,True,True,True,False,True,True,True,True,...,None,None,Firmy,262 KM,None,Niemcy,None,None,Tak,Diesel,Tak,None,None,None,None,None,None,Cayenne,3 000 cm3,None,None,None,None,None,None,None,None,None,Czarny,None,None,Używane,II (2010-),None,None,None,None,None,None,3
4,"[Osobowe, Peugeot, 308, Peugeot 308 T7 (2008-2...","08:46, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",19500.0,"Limanowa, limanowski, Małopolskie",None,Osoba prywatna,False,False,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,False,False,...,None,Tak,Osoby prywatnej,110 KM,None,Polska,None,None,Tak,Diesel,None,None,None,None,None,None,None,308,1

In [19]:
df.shape

(106494, 155)

In [0]:
df = df[df['price_currency'] != 'EUR']

# FEATURE ENGINEERING

In [75]:
SUFFIX_CAT = '__cat'

for feat in df.columns:

  if isinstance(df[feat][0],list): continue

  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:

    df[feat] = factorized_values
  
  else:
    df[feat + SUFFIX_CAT] = factorized_values

cat_feats = [x for x in df.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)


151

#Decision TREE

In [0]:
def run_model(model, feats=cat_feats):

  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)

In [27]:
run_model(DecisionTreeRegressor(max_depth=5))

(-19650.45289201444, 72.79949702628343)

#RANDOM FOREST

In [29]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model)

(-18888.885229877447, 52.00741669248174)

In [31]:
model = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
run_model(model)

[17:08:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:08:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:09:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13011.117848647147, 66.77416696652178)

In [34]:
m= xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)
imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[17:16:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1286 ± 0.0010,param_napęd__cat
0.1113 ± 0.0031,param_rok-produkcji__cat
0.1026 ± 0.0020,param_stan__cat
0.0597 ± 0.0021,param_faktura-vat__cat
0.0523 ± 0.0018,param_moc__cat
0.0440 ± 0.0015,param_skrzynia-biegów__cat
0.0277 ± 0.0005,feature_kamera-cofania__cat
0.0243 ± 0.0007,param_marka-pojazdu__cat
0.0241 ± 0.0004,param_typ__cat
0.0167 ± 0.0007,param_pojemność-skokowa__cat


In [0]:
feats = ['param_napęd__cat', 'param_rok-produkcji','param_stan__cat','param_faktura-vat__cat','param_moc__cat','param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat','param_pojemność-skokowa__cat','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat','param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']

In [36]:
len(feats)

20

In [39]:
run_model(xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0),feats)

[18:35:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13574.091621784022, 118.63492342760455)

In [67]:
df['param_napęd'].value_counts()

Na przednie koła                 66012
Na tylne koła                     8608
4x4 (stały)                       6956
4x4 (dołączany automatycznie)     6876
4x4 (dołączany ręcznie)           1632
Name: param_napęd, dtype: int64

In [76]:
df['param_rok-produkcji__cat'].value_counts()

4     11756
7      7490
11     7305
10     6938
3      6242
      ...  
71        1
66        1
62        1
73        1
69        1
Name: param_rok-produkcji__cat, Length: 77, dtype: int64

In [69]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )

In [53]:
run_model(xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0),feats)

[18:42:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:42:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:42:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11320.72396335277, 98.94067408981262)

In [71]:
df['param_moc'].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [0]:
df['param_moc']= df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )

In [0]:
feats = ['param_napęd__cat', 'param_rok-produkcji','param_stan__cat','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat','param_pojemność-skokowa__cat','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat','param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']

In [78]:
run_model(xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0),feats)

[18:53:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:53:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:53:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9772.408396613564, 85.3312046882783)

In [79]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa']= df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')) )

In [82]:
df['param_pojemność-skokowa']

0          898
2         1560
3         3000
4         1560
5         1984
          ... 
160609    1120
160610    1200
160611    1900
160614    2500
160615    1360
Name: param_pojemność-skokowa, Length: 106494, dtype: int64

In [83]:
feats = ['param_napęd__cat', 'param_rok-produkcji','param_stan__cat','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat','param_pojemność-skokowa','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat','param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']
run_model(xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0),feats)

[18:55:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:55:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9618.08167109244, 79.8982870999676)